In [ ]:
#Dependencies
#@title Import libraries
import os
import time
import tqdm
import torch
import IPython
import torchvision

import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from torchvision.models import AlexNet
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder

from PIL import Image
from io import BytesIO

In [ ]:
# title helper functions
def extract_subimg(im, rot_center, width, height, theta=0):
  rot_bbox = rotbbox_from_coords(rot_center, width, height, theta)

  subimg = im[rot_bbox[0,1]:rot_bbox[1,1],rot_bbox[0,0]:rot_bbox[1,0]]
  rotated_im = ndimage.rotate(subimg, np.degrees(theta)+180)
  newcenter = (np.array(rotated_im.shape)/2).astype(np.int)
  rotated_im = rotated_im[int(newcenter[0]-height/2):int(newcenter[0]+height/2), int(newcenter[1]-width/2):int(newcenter[1]+width/2), :3]  #drop alpha channel, if it's there

  return rotated_im

def extract_subimg_bbox(im, bbox):
  return extract_subimg(im, *unpack_bbox(bbox))

  rot_center = np.array((bbox[1], bbox[0])).T
  width = bbox[3]
  height = bbox[2]
  # theta = -bbox[4]+np.pi/2 #radians --> to include for oriented bounding boxes
  return rot_center, width, height
  #, theta


def rotcorners_from_coords(rot_center, width, height, theta=0):
  rotation = np.array(( (np.cos(theta), -np.sin(theta)),
               (np.sin(theta),  np.cos(theta)))) #--> for oriented bounding box

  wvec = np.dot(rotation, (width/2, 0))
  hvec = np.dot(rotation, (0, height/2))
  
  # There might be a problem
  # corner_points = rot_center + [wvec+hvec, wvec-hvec, -wvec+hvec, -wvec-hvec] #error point
  # corner_points = 
  return corner_points


def rotbbox_from_coords(rot_center, width, height, theta=0):
  corner_points = rotcorners_from_coords(rot_center, width, height, theta)
  rot_bbox = np.array((corner_points.min(0), corner_points.max(0))).astype(np.int)
  #constrain inside image
  rot_bbox[rot_bbox < 0] = 0

  return rot_bbox

def unpack_bbox(bbox):
  #bbox as in the json/COCO data format (centerx, centery, width, height, theta is in radians)
  return bbox

In [ ]:
#@title figure settings

import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/content-creation/main/nma.mplstyle")

plt.rcParams["mpl_toolkits.legacy_colorbar"] = False
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

from matplotlib import pyplot as plt
from matplotlib import rcParams, gridspec
from matplotlib import patches, transforms as plt_transforms

rcParams['figure.figsize'] = [16, 6]
rcParams['font.size'] =14
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [ ]:
#@title set seed

In [ ]:
#@title set device

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#title downloading data

# if not os.path.exists('train_images/'):
#   os.mkdir('train_images/')

#opening an image and displaying!
# path = "/content/gdrive/MyDrive/datasets/efficient_dota/train/images/P0000.png"
path = "/content/gdrive/MyDrive/datasets/efficient_dota/train/P0000.png"



with open(path, 'rb') as my_img:

  img = Image.open(my_img)
  display(img)




In [ ]:
# Working
#Load the json file with the annotations

import json

# datapath = "/content/gdrive/MyDrive/datasets/efficient_dota/train"
datapath = "/content/gdrive/MyDrive/datasets/efficient_dota"
# with open(os.path.join(datapath, 'DOTA_1.0.json')) as f:
with open(os.path.join(datapath, 'annotations/train.json')) as f:
  data = json.load(f)

print(data.keys())
print(data['images'][1])
print(data['annotations'][1])
ann = data['annotations'][1]
print(ann['bbox'][3])

dict_keys(['info', 'images', 'categories', 'annotations'])
{'file_name': 'P0593.png', 'id': 2, 'width': 1434, 'height': 883}
{'area': 613.0, 'category_id': 6, 'segmentation': [[3445, 3391, 3484, 3409, 3478, 3422, 3437, 3402]], 'iscrowd': 0, 'bbox': [3437, 3391, 47, 31], 'image_id': 1, 'id': 2}
31


In [ ]:
#note: ran for 4mins and yes, it's working!

from skimage import io

#Load the images, and make some helpful dict to map the data
imgdir = os.path.join(datapath, 'train')

#remap images to dict by id
imgdict = {l['id']:l for l in data['images']}
#read in all images, can take some time
for i in imgdict.values():
  i['image'] = io.imread(os.path.join(imgdir, i['file_name']))[:, :,: 3]  # drop alpha channel, if it's there

# remap annotations to dict by image_id
from collections import defaultdict
annodict = defaultdict(list)
for annotation in data['annotations']:
  annodict[annotation['image_id']].append(annotation)

# setup list of categories
categories = data['categories']
ncategories = len(categories)
cat_ids = [i['id'] for i in categories]

category_names = {1:'plane',2:'ship',3:'storage tank',4:'baseball diamond',
                  5:'tennis court',6:'basketball court',7: 'ground track field',
                  8:'harbor',9:'bridge',10:'large vehicle',11:'small vehicle',
                  12:'helicopter',13:'roundabout',14:'soccer ball field',
                  15:'swimming pool'}

##Be carefull here


In [ ]:
# Let's look at one image and it's associated annotations

imageid = 74 #<-- Change to show different image with bounding box
im = imgdict[imageid]['image']

#A grid layout to place subplots within a figure.
gs = gridspec.GridSpec(1, 1 + len(annodict[imageid]),
                       width_ratios=[1,]+[.1]*len(annodict[imageid]),
                       wspace=.05)


plt.figure()
ax = plt.subplot(gs[0])
plt.imshow(im)


cmap_normal = plt.Normalize(0, ncategories)

legendList = [,]

for i, annotation in enumerate(annodict[imageid]):
  bbox = annotation['bbox']

  # plt.scatter(*rot_center)
  # plt.scatter(*corner_points.T, c='r')

  ax = plt.subplot(gs[0])
  
  color = plt.cm.jet(cmap_normal(annotation['category_id']))
  # legendDic = {
  #     color:
  #     annotation['category_id']
  # }

  legendList.append(color,annotation['category_id'])

  # print(legendDic)


  
  # ax.legend()
  # print(annotation['category_id'])
  # print(color)
  #bbox mentioned as (a,b,c,d)
  #Rectange(anchor point, width,height,angle)
  # rect = patches.Rectangle([bbox[0] - bbox[2]/2 ,bbox[1] - bbox[3]/2], bbox[3], bbox[2],
  #                          linewidth=1, edgecolor=color, facecolor='none')
  # rect = patches.Rectangle([bbox[0] - bbox[2]/2 ,bbox[1] - bbox[3]/2], bbox[3], bbox[2],
  #                         linewidth=1, edgecolor=color, facecolor='none')
  
 #tugsad

  rect=patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],  linewidth=1, edgecolor=color, facecolor='none')
  
  # t = plt_transforms.Affine2D().rotate_around(bbox[1], bbox[0], -bbox[3]+np.pi/2) --> for rotations (Mostly even the helper functions)

  # rect.set_transform(t + plt.gca().transData)
  ax.add_patch(rect)
  

''' To get the subimage of one object from the overall image. some error while running helper functions '''
  # plt.subplot(gs[i + 1])
  # rotated_im = extract_subimg_bbox(im, bbox)
  # plt.imshow(rotated_im)
  # plt.axis('off')
  # plt.title(annotation['category_id'])
''' Error in plotting bounding boxes. logic of plotting to be checked '''
#do not mention categories like this. mention it as a number near to the object and have a legend in the graph to say what that num means
# plt.colorbar(ticks=range(ncategories), label='category')
# plt.clim(-0.5, ncategories - .5)
# plt.show()


print(legendList)